<h1 align=center> MilliQan Uproot Tutorial</h1>

This jupyter notebook was developed for the 6th MilliQan conference as a tutorial for using the MilliQan data and analysis framework. The framework has been built using the uproot package. Uproot allows root files to be read in as either awkward arrays, numpy arrays, or pandas dataframes. This facilitates faster columnar analysis on the data.

For more information on uproot see the documentation and a tutorials given by J Pivarski and Mason Proffitt below.

[1] https://uproot.readthedocs.io/en/latest/ <br>
[2] https://github.com/jpivarski-talks/2021-06-14-uproot-awkward-columnar-hats/blob/main/2-uproot.ipynb <br>
[3] https://masonproffitt.github.io/uproot-tutorial/aio/index.html

For more information on awkward arrays see the awkward array documentation and tutorial by J Pivarski below.

[1] https://awkward-array.org/doc/main/user-guide/index.html <br>
[2] https://github.com/jpivarski-talks/2021-06-14-uproot-awkward-columnar-hats/blob/main/3-awkward-array.ipynb


## Useful Documents

### Channel Mapping
<img src="https://github.com/carriganm95/milliqanOffline/blob/tutorial/Run3Detector/analysis/tutorial/channelMapping.png?raw=1" alt="Alternative text" />

### Offline Branches: 
https://docs.google.com/document/d/1W-IhqTyuc8L9usIJ3deU526K9qoin0g4_5Q0Zc0dv5Q/edit 

## Importing Packages

The first step will be to import all packages we need. The root and uproot packages are both necessary for our purposes. These need to be installed into this workspace.

To load our files with uproot we also need at least one of awkward, numpy, or pandas. We can easily switch between these different formats and use different packages for different tasks.

We will also import the hist and matplotlib.pyplot packages to make histograms.

Finally we will use the array package to allow us to fill root histograms.

In [ ]:
import os
if not os.path.exists('root_v6.30.02.Linux-ubuntu22.04-x86_64-gcc11.4.tar.gz'):
  !wget https://root.cern/download/root_v6.30.02.Linux-ubuntu22.04-x86_64-gcc11.4.tar.gz
if not os.path.exists('root/bin/thisroot.sh'):
  !tar -xzvf root_v6.30.02.Linux-ubuntu22.04-x86_64-gcc11.4.tar.gz
!source root/bin/thisroot.sh
!pip install uproot
!pip install hist

In [ ]:
import sys
sys.path.append("/content/root/lib")

import ROOT as r
import uproot

import hist
import matplotlib.pyplot as plt

import awkward as ak
import numpy as np
import pandas as pd

import array as arr

## Opening File with Uproot

First we need to download our root file, we will use run 1190 which contains beam-on data. Then we can open our root file and load in the correct tree the same way we do with traditional root.

In [ ]:
if not os.path.isfile('MilliQan_Run1190_v34.root'):
    print("Downloading our example file...")
    filePath = 'https://cernbox.cern.ch/s/7aVy5emV9tivPrl/download'
    os.system("wget -O $PWD/MilliQan_Run1190_v34.root {0} ".format(filePath))
else:
    print("We already have our file!")


In [ ]:
upfile = uproot.open("MilliQan_Run1190_v34.root") #load the root file
uptree = upfile["t"] #open the tree "t"
uptree.show() #print out all branches

Note that in the above print out showing our root file there is also a list of types. There are some types that are vectors and others that are not. For most python dataframes this would cause an issue (numpy, pandas), however the awkward array package was designed to solve this issue. In the same event we can have arrays of different sizes saved.

In [ ]:
#Optionally we can load in the tree at the same time we load the file by appending ":\<treename>\" to the end of the file

uptree = uproot.open("MilliQan_Run1190_v34.root:t")

There are some other useful functions for looking at the contents of our tree.

In [ ]:
uptree.keys() #print the branches

In [ ]:
uptree.typenames() #print branches and type

### Selecting Branches

We can also select specific branches by using the arrays method of uproot. Note that in the below example we can also create a cut, set the library to use, and determine how many events to load.

library = data format to load branches into. Options are awkward (ak, default), pandas (pd), and numpy (np). <br>
cut = string making a cut on the data, cut is applied at this time <br>
entry_stop = sets the number of events to load <br>

In [ ]:
#using numpy

branches = uptree.arrays(["height", "area"], library="np", entry_stop=1) #note that making a cut here cannot be done
branches #print numpy array

In [ ]:
#using pandas

branches = uptree.arrays(["height", "area"], library="pd", entry_stop=10) #note that making a cut here cannot be done
branches #print pandas dataframe

In [ ]:
#using awkward

branches = uptree.arrays(["height", "area"], cut="height >= 500", entry_stop=100000)
branches[:10] #print first 10 events

We can also make more cuts by defining a cut and then applying it to our branches. The cut will be arrays of booleans for each pulse that we can then apply to our data to "make the cut".

In [ ]:
area_cut = branches['area'] >= 500 #cut requires area >= 500 pulse by pulse
height_cut = branches['height'] >= 700 #cut requires height >= 700 pulse by pulse
area_cut #print the area cut

In [ ]:
branches_cut = branches[area_cut] #here we make the cut by saving only those pulses that have a True value for the cut
branches_cut #print

We can make a quick plot of this data by using the hist package. First we need to define our histogram (bins, start, stop) the same as in root. Then we need to fill the histogram with a flattened awkward array.

In [ ]:
h1 = hist.Hist.new.Reg(1000, 0, 200000, name="Area").Double()
h1.fill(ak.flatten(branches_cut['area']))

In [ ]:
branches_cut = branches[area_cut & height_cut] #here we apply both the area and height cuts with & (can also use either with |, and not with ~)
branches_cut #print

In [ ]:
h1 = hist.Hist.new.Reg(1000, 0, 200000, name="Area").Double()
h1.fill(ak.flatten(branches_cut['area']))

## Plotting

### Hist Package

The hist package is designed to quickly make plots in jupyter notebooks.

In [ ]:
#Define a histogram
h1 = hist.Hist.new.Reg(130, 0, 1300, name="Height").Double()
h1.fill(ak.flatten(branches['height']))

### MatPlotlib

We can also use matplotlib plots to take advantage of python tools for plotting.

In [ ]:
plt.hist(ak.flatten(branches['height']), bins=140, range=(0, 1400))
plt.show()

### Root Histograms

Finally we can still use root histograms by converting our entries to array objects. This method of plotting is a bit slow but allows us to use the traditional root tools and save our histograms to root files.

In [ ]:
#define root histogram
h_height = r.TH1F("h_height", "Height", 140, 0, 1400)

#create array from awkward array
heights = arr.array('d', ak.flatten(branches['height']))

#fill the histogram
#inputs are number of entries, values, and weights
h_height.FillN(len(heights), heights, np.ones(len(heights)))

#create a TCanvas to draw on
c1 = r.TCanvas("c1", "c1", 500, 400)

c1.cd()
h_height.Draw()
c1.Draw()

## Iterating Over Multiple Files

The equivalent of TChains is running with uproot iterate. This is what most of the analysis framework is based on.

In [ ]:
#define total events
totalEvents = 0 #count total events
passingEvents = 0 #count events passing cuts

#define histograms
h_npulses = r.TH1F("h_npulses", "Number of Pulses", 50, 0, 50)
h_npulsesCut = r.TH1F("h_npulsesCut", "Number of Pulses After Cuts", 50, 0, 50)

#define TCanvas for plotting
c = r.TCanvas("c1", "c1", 1000, 800)

#use the uproot iterate method to loop over all files in our filelist
#https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.iterate.html
for events in uproot.iterate(

        #list of files to run over
        ["MilliQan_Run1190_v34.root"],

        #branches defined above
        ["area", "height", "npulses", "chan", "type"],

        #if we want to make a cut we can define it here
        #note cannot do this with jagged array (ex event and pulse level quantities)
        cut="height>=0",

        #library used to load data
        #default is awkward array (ak), can also use pandas (pd) or numpy (np)
        library = 'ak',

        #number of events or size per batch
        step_size=1000,

        #number of processors
        num_workers=8

    ):

        totalEvents += len(events)

        #next we can make some cuts on the data
        #ex veto events with hits in top panels
        panel_cut1 = ak.any(events[:, 'chan'] == 68, axis=1) #check if any pulse in event is channel 68 (front top), axis=1 to check for any in event
        panel_cut2 = ak.any(events[:, 'chan'] == 69, axis=1) #back top panel

        passing = events[~panel_cut1 & ~panel_cut2]
        passingEvents += len(passing)

        npulses_pre = ak.count(events['npulses'], axis=1) #count number of pulses in events, note cannot use npulses because some events have empty arrays due to cuts
        npulses_post = ak.count(passing['npulses'], axis=1) #count number of pulses after panel cut

        #turn awkward arrays into arrays
        npulses_pre = arr.array('d', npulses_pre.to_list())
        npulses_post = arr.array('d', npulses_post.to_list())

        #fill histograms
        if len(npulses_pre) > 0: h_npulses.FillN(len(npulses_pre), npulses_pre, np.ones(len(npulses_pre)))
        if len(npulses_post) > 0: h_npulsesCut.FillN(len(npulses_post), npulses_post, np.ones(len(npulses_post)))

print("{0} events passing cuts out of {1} total".format(passingEvents, totalEvents))




Make plot showing the number of pulses before and after cutting on top pannels.

In [ ]:
c.cd()

r.gStyle.SetOptStat(0000) #remove stats box

h_npulses.GetYaxis().SetTitle("Number of Events")
h_npulses.GetXaxis().SetTitle("Number of Pulses")
h_npulses.SetFillColorAlpha(4, 0.3)
h_npulses.Draw()
h_npulsesCut.SetLineColor(2)
h_npulsesCut.SetFillColorAlpha(2, 0.3)
h_npulsesCut.Draw("same")

l1 = r.TLegend(0.5, 0.6, 0.8, 0.7)
l1.AddEntry(h_npulses)
l1.AddEntry(h_npulsesCut)
l1.Draw()

c.Draw()

## MilliQan Processor

To simplify the offline analysis we have designed some tools to work with our data and the uproot packages. The purpose is to centralize the code we use so that we can develop and use the same tools.

We will need to load in the necessary packages from the analysis/utilities/ directory in order to take full advantage of the milliqan analysis framework. The tools we will use are 

milliqanProcessor: Essentially a wrapper for uproot iterate method. Runs over the given files list and applies all cuts, makes plots, and runs custom functions.

milliqanScheduler: Creates schedule of cuts and plots to apply in the processor.

milliqanCuts: Class defining generic cuts, centralize how we define our cuts. 

milliqanPlotter: Class for defining plots.

In [ ]:
import os
import sys
sys.path.append(os.getcwd() + '/../utilities/')
from milliqanProcessor import *
from milliqanScheduler import *
from milliqanCuts import *
from milliqanPlotter import *

%jsroot on

%load_ext autoreload
%autoreload 2

Similar to in uproot iterate we need to define a filelist that we want to run over and define the branches that we require to do our analysis.

In [ ]:
#define a file list to run over
filelist = ['MilliQan_Run1190_v34.root:t']

#define the necessary branches to run over
branches = ['boardsMatched', 'time_module_calibrated', 'height', 'area', 'column', 'row', 'layer']

Next we want to define our cuts object and add all of the cuts we want to make for our analysis. 

The cuts can either come directly from the milliqanCuts class or they can be modifications of those functions using the "getCut" method. This method creates a lambda function of the existing cuts but allows us to change variables in the existing functions.

Inputs for getCut are: <br>

1. original cut function <br>
2. name for the new branch <br>
3. cut value(s) <br>
    

In [ ]:
#define the milliqan cuts object
mycuts = milliqanCuts()

#example of changing cut value
heightCut200 = mycuts.getCut(mycuts.heightCut, 'heightCut200', cut=500)

#example of cutting on events
fourLayerCut = mycuts.getCut(mycuts.fourLayerCut, 'fourLayerCut', cut=False)

It is also possible to create a cut that combined multiple other cuts using the combineCuts function as the original cut function. When using this option the third argument is a list of the branches to combine.

In [ ]:
#example of creating a combined cut
eventCuts = mycuts.getCut(mycuts.combineCuts, 'eventCuts', ['fourLayerCut', 'straightLineCut', 'heightCut200'])

Now we can define the plots to make.

First we define the milliqanPlotter object that will be passed to the milliqanProcessor.

Next we need to define the root histograms that we want to be created.

Finally we need to add the histogram to our plotter using the addHistograms function. This functions takes as inputs <br>

1. the root histogram <br>
2. the branch to be plotted <br>
3. any cuts to be applied before plotting (optional) <br>

In [ ]:
#define milliqan plotter
myplotter = milliqanPlotter()

#create root histogram 
h_height = r.TH1F("h_height", "Height", 140, 0, 1400)

#add root histogram to plotter
myplotter.addHistograms(h_height, 'height', 'eventCuts')

We can now define the order that we would like to create our cuts and plots. We do this by defining a list of both cuts and plots. 

Then we can create a milliQanScheduler object and pass our cutflow to the scheduler.

If we want to inspect our schedule we can also print it out.

In [ ]:
#defining the cutflow
cutflow = [mycuts.layerCut, mycuts.fourLayerCut, mycuts.straightLineCut, heightCut200, eventCuts, myplotter.dict['h_height']]

#create a schedule of the cuts
myschedule = milliQanScheduler(cutflow, mycuts, myplotter)

#print out the schedule
myschedule.printSchedule()

Finally we can define our milliqanProcessor object!

 We need to pass this object:
 1. List of files to run over
 2. List of necessary input branches from root files
 3. The schedule we created
 4. The cut object we created
 5. The plotter object we created
 
 Optionally we can also pass: <br>
 1. max events to run over

In [ ]:
#create the milliqan processor object
myiterator = milliqanProcessor(filelist, branches, myschedule, mycuts, myplotter, max_events=1000)

We can now also optionally set a "custom function" that will be run inside the iterate loop. This is useful because it gives us access to the iterate method without changing the milliqanProcessor.

In [ ]:
#set custom functions inside iterator if desired
#myiterator.setCustomFunction(getMaxPulseTime)

The last step is to run the processor!

In [ ]:
#run the milliqan processor
myiterator.run()

We can now plot the histograms that we created

In [ ]:
c1 = r.TCanvas("c1", "c1", 400, 400)
c1.cd()
h_height.Draw()
c1.Draw()

## Example Analysis: Finding Beam Muons

As an example analysis we will try to look for beam muons by comparing one run with beam on vs another with beam off. Some initial criteria we need to consider for this is

### Event and Pulse Cuts:
1. Boards Matched: Require that all digitizer boards are matched in the event
2. Pickup Flag: Only use pulses passing the pickup flag cut

### Branches 
1. 4 layers hit: we want there to be hits in all four layers of the detector
2. Straight pulse path: we want to have pulses in a straight line path through the detector
3. Channel pulse selection: there can be multiple pulses in a given channel. We need to decide which to use
4. Bar hits: require all hits that we look at are in bars

We also want to define a new cut function to create a branch for the things we want to plot. Here we want to recreate the plot made in the run 2 milliQan analysis looking at cosmic vs beam muon times between layers 1 and 3 (see below). Therefore we will create a function that calculates the times between pulses in different layers.

<img src="https://github.com/carriganm95/milliqanOffline/blob/updateProcessor/Run3Detector/analysis/tutorial/milliqan_beam_muon_Run2.png?raw=true" alt="Alternative text" />

In [ ]:
#define function to get the time difference between pulses in layer1 and layer3
def getPulseDiff(self):
      times = self.events['timeFit_module_calibrated'][self.events['eventCuts']]
      layer = self.events['layer'][self.events['eventCuts']]

      #require only 4 pulses in the event (possible multiple straight line paths were found)
      count = ak.count(times, keepdims=True, axis=1)
      count = count == 4
      count, times = ak.broadcast_arrays(count, times)
      times = times[count]
      layer = layer[count]

      times0 = times[layer == 0]
      times1 = times[layer == 1]
      times2 = times[layer == 2]
      times3 = times[layer == 3]

      #get time difference between two layers (here 3 and 1)
      t_out = times3-times1
      self.events['timeDiff'] = t_out

### Creating our processor:

Now we want to run on our beam on file: Run 1031

We will select the branches that we need for this analysis and develop the cuts

In [ ]:
#define a file list to run over
filelist = ['MilliQan_Run1031_v34.root:t']

#define the necessary branches to run over
branches = ['pickupFlag', 'boardsMatched', 'timeFit_module_calibrated', 'height', 'area', 'column', 'row', 'layer', 'chan', 'ipulse', 'type']

#define the milliqan cuts object
mycuts = milliqanCuts()

#require pulses are not pickup
pickupCut = mycuts.getCut(mycuts.pickupCut, 'pickupCut', cut=True, branches=branches)

#require that all digitizer boards are matched
boardMatchCut = mycuts.getCut(mycuts.boardsMatched, 'boardMatchCut', cut=True, branches=branches)

#Add our custom function
setattr(milliqanCuts, 'getPulseDiff', getPulseDiff)

#Add four layer cut
fourLayerCut = mycuts.getCut(mycuts.fourLayerCut, 'fourLayerCut', cut=False)

#create our combined cut
eventCuts = mycuts.getCut(mycuts.combineCuts, 'eventCuts', ['fourLayerCut', 'straightPulseCut', 'firstChanPulse', 'barCut'])

#define milliqan plotter
myplotter = milliqanPlotter()

#create root histogram 
h_timeDiff_beamOn = r.TH1F("h_timeDiff_beamOn", "Time Diff", 80, -40, 40)

#add root histogram to plotter
myplotter.addHistograms(h_timeDiff_beamOn, 'timeDiff')

#defining the cutflow
cutflow = [boardMatchCut, pickupCut, mycuts.layerCut, mycuts.fourLayerCut, mycuts.straightLineCut, mycuts.firstChanPulse, mycuts.barCut, eventCuts, mycuts.getPulseDiff, myplotter.dict['h_timeDiff_beamOn']]

#create a schedule of the cuts
myschedule = milliQanScheduler(cutflow, mycuts, myplotter)

#print out the schedule
myschedule.printSchedule()

#create the milliqan processor object
myiterator = milliqanProcessor(filelist, branches, myschedule, mycuts, myplotter)

#run the milliqan processor
myiterator.run()

In [ ]:
#check out output histogram

c1 = r.TCanvas("c1", "c1", 400, 400)
c1.cd()
h_timeDiff.Draw()
c1.Draw()

Now we will repeat the same process but with the beam off file: Run 1035

In [ ]:
#define a file list to run over
filelist = ['MilliQan_Run1035_v34.root:t']

#define the necessary branches to run over
branches = ['pickupFlag', 'boardsMatched', 'timeFit_module_calibrated', 'height', 'area', 'column', 'row', 'layer', 'chan', 'ipulse', 'type']

#define the milliqan cuts object
mycuts = milliqanCuts()

#require pulses are not pickup
pickupCut = mycuts.getCut(mycuts.pickupCut, 'pickupCut', cut=True, branches=branches)

#require that all digitizer boards are matched
boardMatchCut = mycuts.getCut(mycuts.boardsMatched, 'boardMatchCut', cut=True, branches=branches)

#Add our custom function
setattr(milliqanCuts, 'getPulseDiff', getPulseDiff)

#Add four layer cut
fourLayerCut = mycuts.getCut(mycuts.fourLayerCut, 'fourLayerCut', cut=False)

#create our combined cut
eventCuts = mycuts.getCut(mycuts.combineCuts, 'eventCuts', ['fourLayerCut', 'straightPulseCut', 'firstChanPulse', 'barCut'])

#define milliqan plotter
myplotter = milliqanPlotter()

#create root histogram 
h_timeDiff_beamOff = r.TH1F("h_timeDiff_beamOff", "Time Diff", 80, -40, 40)

#add root histogram to plotter
myplotter.addHistograms(h_timeDiff_beamOff, 'timeDiff')

#defining the cutflow
cutflow = [boardMatchCut, pickupCut, mycuts.layerCut, mycuts.fourLayerCut, mycuts.straightLineCut, mycuts.firstChanPulse, mycuts.barCut, eventCuts, mycuts.getPulseDiff, myplotter.dict['h_timeDiff_beamOff']]

#create a schedule of the cuts
myschedule = milliQanScheduler(cutflow, mycuts, myplotter)

#print out the schedule
myschedule.printSchedule()

#create the milliqan processor object
myiterator = milliqanProcessor(filelist, branches, myschedule, mycuts, myplotter)

#run the milliqan processor
myiterator.run()

In [ ]:

h1_scaled = h_timeDiff_beamOn.Clone()
scale1 = h1_scaled.Integral(20, 60)
if scale1>0: h1_scaled.Scale(1/scale1)

h_timeDiff_beamOff.SetLineColor(2)
h2_scaled = h_timeDiff_beamOff.Clone()
scale2 = h2_scaled.Integral(20, 60)
if scale2>0: h2_scaled.Scale(1/scale2)

h1_scaled.Draw("hist")
h2_scaled.Draw("hist sames")

c1.Draw()

print("Beam On: \n\tEntries: {0}\n\tMean: {1} +/- {2}\n\tstd: {3}".format(h_timeDiff_beamOn.GetEntries(), h_timeDiff_beamOn.GetMean(), h_timeDiff_beamOn.GetMeanError(), h_timeDiff_beamOn.GetStdDev()))
print("Beam Off: \n\tEntries: {0}\n\tMean: {1} +/- {2}\n\tstd: {3}".format(h_timeDiff_beamOff.GetEntries(), h_timeDiff_beamOff.GetMean(), h_timeDiff_beamOff.GetMeanError(), h_timeDiff_beamOff.GetStdDev()))